# IoT Data Ingestion
Load JSON files from Volume into Delta Table


In [1]:
%run ./data_store_variables


📋 DATA CONFIGURATION VARIABLES
Project Name:        Telecom_IoT_Dashboard
Databricks User:     kunal.gaurav@databricks.com
Clean Username:      kunal-gaurav
Catalog:             kunal
Schema:              telcom
Volume Name:         raw_data
Volume Path:         /Volumes/kunal/telcom/raw_data
Table Name:          kunal.telcom.iot_data_synched_cont

🗄️  LAKEBASE CONFIGURATION VARIABLES
Instance Name:       kunal-gaurav-lakebase-instance
Instance Capacity:   CU_1
Lakebase Catalog:    pg_lakebase_kunal-gaurav
Synced Table:        kunal.telcom.iot_data_synced
PG Table:            kunal.telcom.iot_metadata
✅ Catalog 'kunal' ready
✅ Schema 'kunal.telcom' ready
✅ Volume '/Volumes/kunal/telcom/raw_data' ready
✅ Table 'kunal.telcom.iot_data_synched_cont' ready


In [2]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType, TimestampType

schema = StructType() \
    .add("tower_id", StringType()) \
    .add("region", StringType()) \
    .add("timestamp", TimestampType()) \
    .add("data_usage_mb", DoubleType()) \
    .add("call_drop_rate", DoubleType()) \
    .add("active_users", IntegerType())

# Volume path for IOT logs
volume_path = f"{VOLUME_PATH}/IOT_LOGS"
print(volume_path)
# Checkpoint location
checkpoint_path = f"{VOLUME_PATH}/checkpoint"

# Read from JSON files (simulated streaming)
stream_df = (spark.readStream
    .format("cloudFiles")
    .schema(schema)
    .option("cloudFiles.format", "json")
    .load(volume_path))

query = (stream_df
 .writeStream
 .format("delta")
 .outputMode("append")
 .option("checkpointLocation", checkpoint_path)
 .trigger(processingTime="10 seconds")
 .queryName("iot_data_streaming")  # Named query
 .table(TABLE_NAME)
)

print(f"✅ Streaming started: {query.name}")
print(f"📊 Query ID: {query.id}")

/Volumes/kunal/telcom/raw_data/IOT_LOGS
✅ Streaming started: iot_data_streaming
📊 Query ID: e6a380b6-8147-4717-9871-a614e82cf44e


In [7]:
# Get query stats
query_status = query.status

print("=" * 60)
print("📊 STREAMING QUERY STATUS")
print("=" * 60)
print(f"Query Name: {query.name}")
print(f"Query ID: {query.id}")
print(f"Is Active: {query.isActive}")
print(f"Message: {query_status['message']}")
print(f"Data Available: {query_status['isDataAvailable']}")
print(f"Trigger Active: {query_status['isTriggerActive']}")
print("=" * 60)

# Get recent progress
recent_progress = query.recentProgress
if recent_progress:
    latest = recent_progress[-1]
    print("\n📈 LATEST PROGRESS:")
    print(f"Batch ID: {latest.get('batchId', 'N/A')}")
    print(f"Input Rows: {latest.get('numInputRows', 0)}")
    print(f"Processing Time: {latest.get('durationMs', {}).get('triggerExecution', 'N/A')} ms")
    print(f"Timestamp: {latest.get('timestamp', 'N/A')}")

/Users/kunal.gaurav/Documents/Cursor/Telecom IoT_lakebase_aap_dash/.venv/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:2181: UserWarning: Spark Connect Session expired on the server. Please generate a new session by detaching and reattaching the compute if in a Databricks notebook or job or by calling DatabricksSession.builder.getOrCreate() if using Databricks Connect.
  warnings.warn(
/Users/kunal.gaurav/Documents/Cursor/Telecom IoT_lakebase_aap_dash/.venv/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py:253: UserWarning: ReleaseExecute failed with exception: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "[INVALID_HANDLE.SESSION_CLOSED] The handle bb90dfa4-9904-4d8e-9f65-bd303f9fd489 is invalid. Session was closed. SQLSTATE: HY000"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_status:13, grpc_message:"[INVALID_HANDLE.SESSION_CLOSED] The handle bb90dfa4-9904-4d8e-9f65-bd303f9fd489 is invali

SparkConnectGrpcException: (org.apache.spark.SparkSQLException) [INVALID_HANDLE.SESSION_CLOSED] The handle bb90dfa4-9904-4d8e-9f65-bd303f9fd489 is invalid. Session was closed. SQLSTATE: HY000

In [10]:
# Stop the streaming query
try:
    query.stop()
    print(f"⏹️  Streaming query '{query.name}' stopped successfully")
    print(f"   Final status: {query.isActive}")
except Exception as e:
    print(f"❌ Error stopping query: {e}")

# Verify all streaming queries are stopped
active_queries = spark.streams.active
if active_queries:
    print(f"\n⚠️  Still {len(active_queries)} active streaming queries:")
    for q in active_queries:
        print(f"   - {q.name} (ID: {q.id})")
else:
    print("\n✅ No active streaming queries")

/Users/kunal.gaurav/Documents/Cursor/Telecom IoT_lakebase_aap_dash/.venv/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:2181: UserWarning: Spark Connect Session expired on the server. Please generate a new session by detaching and reattaching the compute if in a Databricks notebook or job or by calling DatabricksSession.builder.getOrCreate() if using Databricks Connect.
  warnings.warn(
/Users/kunal.gaurav/Documents/Cursor/Telecom IoT_lakebase_aap_dash/.venv/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py:253: UserWarning: ReleaseExecute failed with exception: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "[INVALID_HANDLE.SESSION_CLOSED] The handle c592c8dc-5c09-4c62-879f-d06ae40bb329 is invalid. Session was closed. SQLSTATE: HY000"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"[INVALID_HANDLE.SESSION_CLOSED] The handle c592c8dc-5c09-4c62-879f-d06ae40bb329 is invalid. Session was c

❌ Error stopping query: (org.apache.spark.SparkSQLException) [INVALID_HANDLE.SESSION_CLOSED] The handle c592c8dc-5c09-4c62-879f-d06ae40bb329 is invalid. Session was closed. SQLSTATE: HY000


/Users/kunal.gaurav/Documents/Cursor/Telecom IoT_lakebase_aap_dash/.venv/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py:253: UserWarning: ReleaseExecute failed with exception: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "[INVALID_HANDLE.SESSION_CLOSED] The handle c592c8dc-5c09-4c62-879f-d06ae40bb329 is invalid. Session was closed. SQLSTATE: HY000"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_status:13, grpc_message:"[INVALID_HANDLE.SESSION_CLOSED] The handle c592c8dc-5c09-4c62-879f-d06ae40bb329 is invalid. Session was closed. SQLSTATE: HY000"}"
>.
  warnings.warn(f"ReleaseExecute failed with exception: {e}.")


SparkConnectGrpcException: (org.apache.spark.SparkSQLException) [INVALID_HANDLE.SESSION_CLOSED] The handle c592c8dc-5c09-4c62-879f-d06ae40bb329 is invalid. Session was closed. SQLSTATE: HY000